***Coursera Week 3 Submission Part-2 (Continued after Part-1)***

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown in the coursera website.

I did mention the steps below as to how I proceeded to create the final pandas Dataframe

***PART 1***


In [1]:
import numpy as np
import json
import geopy
from geopy.geocoders import Nominatim

import folium 
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)

In [2]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header = None)


required_cols = ['Postcode','Borough','Neighbourhood']

for t in tables:
    if(str(np.array_equal(np.array(t.columns),np.array(required_cols))) == "True"):
        pstl_data_df = pd.DataFrame(t)
        break
    print("Shape",pstl_data_df.shape)
    

In [3]:
pstl_data_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## ***Step 2***
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [4]:
# Filter out Boroughs what have not been assigned

pstl_data_df = pstl_data_df[pstl_data_df.Borough != "not assigned"]
print("Shape", pstl_data_df.shape)
pstl_data_df.head()

Shape (287, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## ***Step 3***
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.


In [5]:
pstl_data_df['new_nghbr'] = np.where(pstl_data_df['Neighbourhood'] == 'Not assigned',pstl_data_df['Borough'],pstl_data_df['Neighbourhood'])
pstl_data_df.head(15)

,Postcode,Borough,Neighbourhood,new_nghbr
0,M1A,Not assigned,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned,Not assigned
2,M3A,North York,Parkwoods,Parkwoods
3,M4A,North York,Victoria Village,Victoria Village
4,M5A,Downtown Toronto,Harbourfront,Harbourfront
5,M6A,North York,Lawrence Heights,Lawrence Heights
6,M6A,North York,Lawrence Manor,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park,Queen's Park
8,M8A,Not assigned,Not assigned,Not assigned
9,M9A,Etobicoke,Islington Avenue,Islington Avenue


## ***Step 4*** 

More than one neighborhood can exist in one postal code area.For example, in the table on the Wikipedia page, you will notice that M5A is listed twice ad has two neighborhoods:
- Harbour Front
- Regent Park

The two rows above need to be combined into a single row
(The neighbour hoods needs to be separated with a comma 

In [6]:
## Hint -> Use new_nghbor column to get a grouped column
# Whenever the same Postal Code and the same Borough are present we
# have to club the Neighborhood Column using the functions
# groupby and apply


In [7]:
can_postal_cd_df = pd.DataFrame(pstl_data_df.groupby(['Postcode','Borough'])['new_nghbr'].apply(','.join).reset_index())

can_postal_cd_df.head()

,Postcode,Borough,new_nghbr
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Rouge,Malvern"
2,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
3,M1E,Scarborough,"Guildwood,Morningside,West Hill"
4,M1G,Scarborough,Woburn


In [8]:
# Renaming columns to the same on in the image given:
can_postal_cd_df = can_postal_cd_df.rename(columns = {"Postcode": "PostalCode","new_nghbr":"Neighborhood"})

print(can_postal_cd_df.head())
print("Final Shape of Data Frame : ", can_postal_cd_df.shape)

  PostalCode       Borough                          Neighborhood
0        M1A  Not assigned                          Not assigned
1        M1B   Scarborough                         Rouge,Malvern
2        M1C   Scarborough  Highland Creek,Rouge Hill,Port Union
3        M1E   Scarborough       Guildwood,Morningside,West Hill
4        M1G   Scarborough                                Woburn
Final Shape of Data Frame :  (180, 3)


In [9]:
can_postal_cd_df.dtypes

PostalCode      object
Borough         object
Neighborhood    object
dtype: object

***As we see above the shape of the new data frame created is (103,3)***

# ***PART-2***

In [8]:
!conda install -c conda-forge geocoder --yes

Solving environment: done

# All requested packages already installed.



In [9]:
import geocoder

def get_geocoder(postal_code_from_df):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code_from_df.strip()))
        lat_lng_coords = g.latlng
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    return latitude,longitude

In [10]:
# Add the lat and long value column to the Pandas df

can_postal_cd_df['Latitude'],can_postal_cd_df['Longitude']= zip(*can_postal_cd_df['PostalCode'].apply(get_geocoder))
can_postal_cd_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A,Not assigned,Not assigned,43.648690,-79.385440
1,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
2,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785665,-79.158725
3,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765815,-79.175193
4,M1G,Scarborough,Woburn,43.768369,-79.217590


In [11]:
print("Shape: ", can_postal_cd_df.shape)


Shape:  (180, 5)


In [12]:
# Map of Toronto



In [13]:
addr = 'Toronto, Ontario'

geolocator = Nominatim(user_agent = "toronto_ontario")
location = geolocator.geocode(addr)
latitude = location.latitude
longitude = location.longitude
print("Geographical coordinates of Toronto, Ontario  are {},{}".format(latitude,longitude))


Geographical coordinates of Toronto, Ontario  are 43.653963,-79.387207


In [14]:
#Folium for mapping


In [15]:
map_toronto = folium.Map(location = [latitude, longitude],zoom_start = 11)

for lat,long,post, borough, neigh in zip(can_postal_cd_df['Latitude'], can_postal_cd_df['Longitude'],can_postal_cd_df['PostalCode'],can_postal_cd_df['Borough'],can_postal_cd_df['Neighborhood']):
    label = "{} ({}): {}".format(borough, post,neigh)
    popup = folium.Popup(label,parse_html = True)
    folium.CircleMarker(
    [lat,long],
    radius = 5,
    popup = popup,
    color = 'green',
    fill = True,
    fill_color = '#3186cc',
    fill_opacity = 0.8,
    parse_html = False).add_to(map_toronto)
    
map_toronto

# PART 3


In [16]:
# Step 1 - Filtering the Toronto Boroughs
can_postal_cd_df.Borough.unique()

array(['Not assigned', 'Scarborough', 'North York', 'East York',
       'East Toronto', 'Central Toronto', 'Downtown Toronto', 'York',
       'West Toronto', 'Mississauga', 'Etobicoke'], dtype=object)

In [17]:
toronto_b = ['East Toronto','Central Toronto','Downtown Toronto','West Toronto']
toronto_df = can_postal_cd_df[can_postal_cd_df['Borough'].isin(toronto_b)].reset_index(drop = True)

In [18]:
print("New Shape: ",toronto_df)
toronto_df.sample(30)

New Shape:     PostalCode           Borough  \
0         M4E      East Toronto   
1         M4K      East Toronto   
2         M4L      East Toronto   
3         M4M      East Toronto   
4         M4N   Central Toronto   
5         M4P   Central Toronto   
6         M4R   Central Toronto   
7         M4S   Central Toronto   
8         M4T   Central Toronto   
9         M4V   Central Toronto   
10        M4W  Downtown Toronto   
11        M4X  Downtown Toronto   
12        M4Y  Downtown Toronto   
13        M5A  Downtown Toronto   
14        M5B  Downtown Toronto   
15        M5C  Downtown Toronto   
16        M5E  Downtown Toronto   
17        M5G  Downtown Toronto   
18        M5H  Downtown Toronto   
19        M5J  Downtown Toronto   
20        M5K  Downtown Toronto   
21        M5L  Downtown Toronto   
22        M5N   Central Toronto   
23        M5P   Central Toronto   
24        M5R   Central Toronto   
25        M5S  Downtown Toronto   
26        M5T  Downtown Toronto   
27      

,PostalCode,Borough,Neighborhood,Latitude,Longitude
36,M6S,West Toronto,"Runnymede,Swansea",43.649620,-79.476141
19,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.630210,-79.362433
3,M4M,East Toronto,Studio District,43.660629,-79.334855
4,M4N,Central Toronto,Lawrence Park,43.728420,-79.387133
5,M4P,Central Toronto,Davisville North,43.712755,-79.388514
32,M6J,West Toronto,"Little Portugal,Trinity",43.648525,-79.417757
10,M4W,Downtown Toronto,Rosedale,43.682205,-79.377945
12,M4Y,Downtown Toronto,Church and Wellesley,43.666585,-79.381302
25,M5S,Downtown Toronto,"Harbord,University of Toronto",43.663110,-79.401801
1,M4K,East Toronto,"The Danforth West,Riverdale",43.683178,-79.355105


In [19]:
#Use Foursquare API 


In [20]:
CLIENT_ID = 'JLONPWEZYTARJ1PN2FLSWHRUBQH5YEBLASLGOCUVR555BJLE'
CLIENT_SECRET = 'SP5RTEEKYS4TFFKDKG0U1I5CPEV2QMELIENUCZ3JBZZBWUID'
VERSION = '20200303'
print('credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

credentails:
CLIENT_ID: JLONPWEZYTARJ1PN2FLSWHRUBQH5YEBLASLGOCUVR555BJLE
CLIENT_SECRET:SP5RTEEKYS4TFFKDKG0U1I5CPEV2QMELIENUCZ3JBZZBWUID


In [22]:
import requests

RADIUS = 500
LIMIT = 1000


recommend = []

for lat,long,post, borough, neigh in zip(toronto_df['Latitude'],toronto_df['Longitude'], toronto_df['PostalCode'], toronto_df['Borough'], toronto_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        RADIUS, 
        LIMIT)
    
    postal_data = requests.get(url).json()["response"]['groups'][0]['items']
    for recommend_post in postal_data:
        recommend.append((
        post,
        borough,
        neigh,
        latitude,
        longitude,
        recommend_post['venue']['name'], 
            recommend_post['venue']['location']['lat'], 
            recommend_post['venue']['location']['lng'],  
            recommend_post['venue']['categories'][0]['name']))    
    

In [23]:
toronto_rec = pd.DataFrame(recommend)
toronto_rec.columns = ['PostalCode', 'Borough', 'Neighborhoods', 'Borough_Lat', 'Borough_Long', 'Venue_Name', 'Venue_Lat', 'Venue_Long', 'Venue_Category']
print("The Shape of Recommendations (Postal Codes in Toronto): ",toronto_rec.shape)


toronto_rec.drop_duplicates(keep = False, inplace = True)

toronto_rec.shape

The Shape of Recommendations (Postal Codes in Toronto):  (1772, 9)


(1772, 9)

In [24]:
toronto_rec.head()

,PostalCode,Borough,Neighborhoods,Borough_Lat,Borough_Long,Venue_Name,Venue_Lat,Venue_Long,Venue_Category
0,M4E,East Toronto,The Beaches,43.653963,-79.387207,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.653963,-79.387207,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.653963,-79.387207,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.653963,-79.387207,Dip 'n Sip,43.678897,-79.297745,Coffee Shop
4,M4E,East Toronto,The Beaches,43.653963,-79.387207,Upper Beaches,43.680563,-79.292869,Neighborhood


In [25]:
toronto_rec.groupby(['PostalCode', 'Borough', 'Neighborhoods'])['Venue_Name'].count()

PostalCode  Borough           Neighborhoods                                                                                       
M4E         East Toronto      The Beaches                                                                                               5
M4K         East Toronto      The Danforth West,Riverdale                                                                               5
M4L         East Toronto      The Beaches West,India Bazaar                                                                            16
M4M         East Toronto      Studio District                                                                                          50
M4N         Central Toronto   Lawrence Park                                                                                             2
M4P         Central Toronto   Davisville North                                                                                          7
M4R         Central Toronto   North Toron

In [26]:
print("Unique Venue Categories are: ",len(toronto_rec['Venue_Category'].unique()))
toronto_rec['Venue_Category'].unique()

Unique Venue Categories are:  220


array(['Trail', 'Health Food Store', 'Pub', 'Coffee Shop', 'Neighborhood',
       'Ice Cream Shop', 'Discount Store', 'Park', 'Bus Line',
       'Grocery Store', 'Gym', 'Fast Food Restaurant',
       'Fish & Chips Shop', 'Italian Restaurant', 'Sushi Restaurant',
       'Liquor Store', 'Pet Store', 'Steakhouse', 'Burrito Place',
       'Movie Theater', 'Sandwich Place', 'Pizza Place', 'Bakery',
       'Latin American Restaurant', 'Vietnamese Restaurant', 'Boutique',
       'Café', 'Brewery', 'Diner', 'Antique Shop', 'Bar',
       'American Restaurant', 'Arts & Crafts Store', 'Gay Bar',
       'Comfort Food Restaurant', 'Flea Market', 'French Restaurant',
       'Thai Restaurant', 'Seafood Restaurant', 'Gastropub',
       'Clothing Store', 'Convenience Store', 'Flower Shop',
       'Light Rail Station', 'Swim School', 'Food & Drink Shop',
       'Breakfast Spot', 'Department Store', 'Hotel', 'Dog Run',
       'Playground', 'Gym Pool', 'Garden', 'Dessert Shop',
       'Indian Restaurant',

In [27]:
# Analyszing Venues by area



In [28]:
toronto_rec = toronto_rec.drop(['Borough_Lat','Borough_Long','Venue_Lat','Venue_Long'],axis=1)

In [31]:
toronto_rec_df = pd.get_dummies(toronto_rec, columns =['Venue_Category'],prefix = "",prefix_sep = "")


In [33]:
toronto_rec_df.head()

,PostalCode,Borough,Neighborhoods,Venue_Name,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,M4E,East Toronto,The Beaches,Glen Manor Ravine,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,The Big Carrot Natural Food Market,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [34]:
toronto_venues_freq = toronto_rec_df.groupby(['PostalCode','Borough','Neighborhoods']).mean().reset_index()
toronto_venues_freq.head()

,PostalCode,Borough,Neighborhoods,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.20,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0000,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# Create a new dataframe
all_toronto_venues = pd.DataFrame(columns=columns)
all_toronto_venues['PostalCode'] = toronto_venues_freq['PostalCode']
all_toronto_venues['Borough'] = toronto_venues_freq['Borough']
all_toronto_venues['Neighborhoods'] = toronto_venues_freq['Neighborhoods']
for ind in np.arange(toronto_venues_freq.shape[0]):
    row_categories = toronto_venues_freq.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    all_toronto_venues.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

all_toronto_venues.sort_values(freqColumns, inplace=True)
all_toronto_venues

,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,M5T,Downtown Toronto,"Chinatown,Grange Park,Kensington Market",Bar,Café,Chinese Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Mexican Restaurant,Dumpling Restaurant,Bakery,Farmers Market
33,M6P,West Toronto,"High Park,The Junction South",Bed & Breakfast,Park,Yoga Studio,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
1,M4K,East Toronto,"The Danforth West,Riverdale",Bus Line,Park,Ice Cream Shop,Grocery Store,Discount Store,Yoga Studio,Electronics Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant
4,M4N,Central Toronto,Lawrence Park,Bus Line,Swim School,Yoga Studio,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
29,M6G,Downtown Toronto,Christie,Café,Grocery Store,Coffee Shop,Playground,Candy Store,Baby Store,Italian Restaurant,Yoga Studio,Event Space,Elementary School
24,M5S,Downtown Toronto,"Harbord,University of Toronto",Café,Restaurant,Coffee Shop,Bakery,Italian Restaurant,Japanese Restaurant,Gym,Bookstore,Bar,Pharmacy
13,M5A,Downtown Toronto,Harbourfront,Coffee Shop,Bakery,Boat or Ferry,Theater,French Restaurant,Breakfast Spot,Park,Ice Cream Shop,Performing Arts Venue,Spa
32,M6K,West Toronto,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Bakery,Restaurant,Café,Furniture / Home Store,Bar,Supermarket,Italian Restaurant,Sandwich Place,Hotel
27,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,Coffee Shop,Bar,Hotel,Sushi Restaurant,Pizza Place,Seafood Restaurant,Café,Restaurant,Steakhouse,Thai Restaurant
37,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Bar,Hotel,Sushi Restaurant,Pizza Place,Seafood Restaurant,Café,Restaurant,Steakhouse,Thai Restaurant


In [38]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 10

toronto_venues_freq_clustering = toronto_venues_freq.drop(['PostalCode', 'Borough', 'Neighborhoods'], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_venues_freq_clustering)

#No data for a particular postal code so removed it
toronto_clustered_df = toronto_df.drop(toronto_df.index[toronto_df.PostalCode=='M5N'])
toronto_clustered_df['Cluster'] = kmeans.labels_

toronto_clustered_df = toronto_clustered_df.join(all_toronto_venues.drop(['Borough', 'Neighborhoods'], 1).set_index('PostalCode'), on='PostalCode')
toronto_clustered_df.sort_values(['Cluster'] + freqColumns, inplace=True)
toronto_clustered_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,M5T,Downtown Toronto,"Chinatown,Grange Park,Kensington Market",43.653530,-79.397233,0,Bar,Café,Chinese Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Mexican Restaurant,Dumpling Restaurant,Bakery,Farmers Market
30,M6G,Downtown Toronto,Christie,43.668781,-79.420710,0,Café,Grocery Store,Coffee Shop,Playground,Candy Store,Baby Store,Italian Restaurant,Yoga Studio,Event Space,Elementary School
25,M5S,Downtown Toronto,"Harbord,University of Toronto",43.663110,-79.401801,0,Café,Restaurant,Coffee Shop,Bakery,Italian Restaurant,Japanese Restaurant,Gym,Bookstore,Bar,Pharmacy
13,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166,0,Coffee Shop,Bakery,Boat or Ferry,Theater,French Restaurant,Breakfast Spot,Park,Ice Cream Shop,Performing Arts Venue,Spa
33,M6K,West Toronto,"Brockton,Exhibition Place,Parkdale Village",43.639410,-79.424362,0,Coffee Shop,Bakery,Restaurant,Café,Furniture / Home Store,Bar,Supermarket,Italian Restaurant,Sandwich Place,Hotel
28,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.648690,-79.385440,0,Coffee Shop,Bar,Hotel,Sushi Restaurant,Pizza Place,Seafood Restaurant,Café,Restaurant,Steakhouse,Thai Restaurant
38,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.648690,-79.385440,0,Coffee Shop,Bar,Hotel,Sushi Restaurant,Pizza Place,Seafood Restaurant,Café,Restaurant,Steakhouse,Thai Restaurant
32,M6J,West Toronto,"Little Portugal,Trinity",43.648525,-79.417757,0,Coffee Shop,Bar,Wine Bar,Restaurant,Asian Restaurant,Cocktail Bar,Pizza Place,Vietnamese Restaurant,Bakery,Men's Store
36,M6S,West Toronto,"Runnymede,Swansea",43.649620,-79.476141,0,Coffee Shop,Café,Bakery,Pizza Place,Gym,Liquor Store,Bookstore,Flower Shop,Soccer Field,Bus Line
20,M5K,Downtown Toronto,"Design Exchange,Toronto Dominion Centre",43.647100,-79.381531,0,Coffee Shop,Café,Hotel,Restaurant,Bakery,Bar,Japanese Restaurant,Italian Restaurant,Seafood Restaurant,Gastropub


In [40]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_clustered_df['Latitude'], toronto_clustered_df['Longitude'], toronto_clustered_df['PostalCode'], toronto_clustered_df['Borough'], toronto_clustered_df['Neighborhood'], toronto_clustered_df['Cluster']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters